In [1]:
import itertools
import pandas as pd
import numpy as np

In [2]:
df = pd.read_parquet("../data/silver/states_dataset.parquet")
df['pump_active'] = ((df['pump_1_active']) | (df['pump_2_active'])).astype(int)
df = df.drop(columns=['pump_1_active', 'pump_2_active', 'pump_1_duration_sum', 'pump_2_duration_sum', 'timestamp'])
df = df.iloc[72:]
initial_state = df.iloc[1000].to_dict()

In [5]:
df

,hour,day_of_week,week_of_year,input_flow_rate_first,input_flow_rate_last,input_flow_rate_mean,reservoir_level_percentage_first,reservoir_level_percentage_last,reservoir_level_percentage_mean,output_flow_rate_first,...,forecast_output_flow_rate_17h,forecast_output_flow_rate_18h,forecast_output_flow_rate_19h,forecast_output_flow_rate_20h,forecast_output_flow_rate_21h,forecast_output_flow_rate_22h,forecast_output_flow_rate_23h,forecast_output_flow_rate_24h,time_to_depletion,pump_active
73,13,0,12,67.30,65.24,65.229474,42.86,48.76,45.783158,43.300000,...,29.052292,31.430012,32.951867,34.134177,33.877160,33.797010,35.812558,36.069254,3,1
74,14,0,12,65.24,65.64,59.777083,49.20,54.20,52.341667,35.906667,...,27.754940,30.306910,30.653275,31.108195,32.442686,33.750374,33.922880,33.039178,3,1
75,15,0,12,65.64,65.24,65.126842,54.20,62.46,59.038947,65.640000,...,39.087879,38.030320,37.767012,37.027046,35.742707,38.602135,37.159135,34.491482,3,1
76,16,0,12,65.24,64.07,64.710909,62.46,70.70,66.544545,65.240000,...,39.777063,42.477820,47.057412,46.689354,46.955455,45.386336,45.397087,61.838079,2,1
77,17,0,12,64.07,0.00,38.829500,71.74,70.20,73.144000,40.958889,...,31.261251,31.075864,31.615369,31.760121,32.158476,32.588876,48.214572,55.097034,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8632,4,0,11,0.00,0.00,0.000000,81.56,74.50,77.947619,16.666667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5,0
8633,5,0,11,0.00,0.00,0.000000,74.16,66.86,70.361000,11.333333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5,0
8634,6,0,11,0.00,69.93,15.126667,66.40,61.30,62.408889,30.666667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5,1
8635,7,0,11,70.39,66.46,67.373500,62.40,72.40,67.343000,33.723333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5,1


In [3]:
df = pd.read_parquet("../data/silver/states_dataset.parquet")
df['pump_active'] = ((df['pump_1_active']) | (df['pump_2_active'])).astype(int)
df = df.drop(columns=['pump_1_active', 'pump_2_active', 'pump_1_duration_sum', 'pump_2_duration_sum', 'timestamp'])
df = df.iloc[72:]
initial_state = df.iloc[1000].to_dict() # change this line to get the initial state from the timestamp created from the year, month, day, hour inputed 

reservoir_capacity = 1000 * 1000  # in liters

POPULATION_SIZE = 100
MUTATION_RATE = 0.1
NUM_GENERATIONS = 100

def calculate_reward(pump_schedule, state):
    rewards = 0
    current_reservoir_level = state['reservoir_level_percentage_mean'] * reservoir_capacity * 0.01

    for hour, pump_status in enumerate(pump_schedule):
        outflow_rate = state[f'forecast_output_flow_rate_{hour + 1}h']
        inflow_rate = state['input_flow_rate_mean'] * 3600  # Convert L/S to L/H

        # Update the reservoir level based on inflow and outflow
        new_reservoir_level = current_reservoir_level + inflow_rate - (outflow_rate * pump_status)

        # Penalize if the reservoir goes below 20% or above 95%
        if new_reservoir_level < 0.2 * reservoir_capacity:
            rewards -= 50
        elif new_reservoir_level > 0.95 * reservoir_capacity:
            rewards -= 50
        else:
            # Reward for keeping the reservoir within the desired range
            rewards += 50

        # Penalize for using the pump during peak hours (18h to 21h)
        if 18 <= (state['hour'] + hour) % 24 <= 21 and pump_status == 1:
            rewards -= 20
        # Reward for using the pump during non-peak hours (0h to 5h)
        elif 0 <= (state['hour'] + hour) % 24 <= 5 and pump_status == 1:
            rewards += 20

        # Reward for maintaining a good time_to_depletion
        if state['time_to_depletion'] < 2:
            rewards -= 20
        elif state['time_to_depletion'] > 10:
            rewards += 100
        elif state['time_to_depletion'] > 6:
            rewards += 50
        elif state['time_to_depletion'] > 4:
            rewards += 20

        # Update the reservoir level
        current_reservoir_level = new_reservoir_level

    return rewards

# Initialize the population with random pump schedules
def initialize_population(size):
    return [np.random.choice([0, 1], size=24) for _ in range(size)]

# Select parents for crossover
def select_parents(population, rewards):
    probabilities = rewards / np.sum(rewards)
    parents_indices = np.random.choice(len(population), size=2, p=probabilities)
    return population[parents_indices[0]], population[parents_indices[1]]

# Perform crossover between two parents
def crossover(parent1, parent2):
    crossover_point = np.random.randint(1, 23)
    child1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
    child2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
    return child1, child2

# Mutate a child's pump schedule
def mutate(child):
    for i in range(len(child)):
        if np.random.rand() < MUTATION_RATE:
            child[i] = 1 - child[i]
    return child

# Run the genetic algorithm
population = initialize_population(POPULATION_SIZE)
best_schedule = None
best_reward = -np.inf

for generation in range(NUM_GENERATIONS):
    rewards = np.array([calculate_reward(individual, initial_state) for individual in population])
    
    if rewards.max() > best_reward:
        best_reward = rewards.max()
        best_schedule = population[rewards.argmax()]
    
    new_population = []
    
    for _ in range(POPULATION_SIZE // 2):
        parent1, parent2 = select_parents(population, rewards)
        child1, child2 = crossover(parent1, parent2)
        new_population.extend([mutate(child1), mutate(child2)])
    
    population = new_population

print(f"Best schedule: {best_schedule}")
print(f"Best reward: {best_reward}")

Best schedule: [1 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 0 1 1 1 1 1 1]
Best reward: -980


In [4]:
POPULATION_SIZE = 100
MUTATION_RATE = 0.1
NUM_GENERATIONS = 1000

initial_state = df.iloc[1000].to_dict()

def calculate_reward(pump_schedule, state):
    rewards = 0
    current_reservoir_level = state['reservoir_level_percentage_mean'] * reservoir_capacity * 0.01

    for hour, pump_status in enumerate(pump_schedule):
        outflow_rate = state[f'forecast_output_flow_rate_{hour + 1}h'] * 3600  # Convert L/S to L/H
        inflow_rate = state['input_flow_rate_mean'] * 3600  # Convert L/S to L/H

        # Update the reservoir level based on inflow and outflow
        new_reservoir_level = current_reservoir_level + (inflow_rate * pump_status) - outflow_rate

        # Penalize if the reservoir goes below 20% or above 95%
        if new_reservoir_level < 0.2 * reservoir_capacity:
            rewards -= 50
        elif new_reservoir_level > 0.95 * reservoir_capacity:
            rewards -= 50
        else:
            # Reward for keeping the reservoir within the desired range
            # Higher reward as it gets closer to 80%, and decreasing reward past 80% until 95%
            if new_reservoir_level <= 0.8 * reservoir_capacity:
                rewards += (new_reservoir_level / (0.8 * reservoir_capacity)) * 50
            else:
                rewards += ((0.95 * reservoir_capacity - new_reservoir_level) / (0.15 * reservoir_capacity)) * 50

        # Penalize for using the pump during peak hours (18h to 21h)
        if 18 <= (state['hour'] + hour) % 24 <= 21 and pump_status == 1:
            rewards -= 20
        # Reward for using the pump during normal hours (6h to 17h)
        elif 6 <= (state['hour'] + hour) % 24 <= 17 and pump_status == 1:
            rewards += 10
        # Reward for using the pump during non-peak hours (0h to 5h)
        elif 0 <= (state['hour'] + hour) % 24 <= 5 and pump_status == 1:
            rewards += 20

        # Reward for maintaining a good time_to_depletion
        if state['time_to_depletion'] == 2:
            rewards -= 20
        elif state['time_to_depletion'] == 1:
            rewards -= 50
        elif state['time_to_depletion'] == 0:
            rewards -= 100
        elif state['time_to_depletion'] > 10:
            rewards += 100
        elif state['time_to_depletion'] > 6:
            rewards += 50
        elif state['time_to_depletion'] > 4:
            rewards += 20

        # Update the reservoir level
        current_reservoir_level = new_reservoir_level

    return rewards

# Initialize the population with random pump schedules
def initialize_population(size):
    return [np.random.choice([0, 1], size=24) for _ in range(size)]

# Select parents for crossover
def select_parents(population, rewards):
    min_reward = np.min(rewards)
    shifted_rewards = rewards - min_reward + 1  # Shift rewards to make them non-negative
    probabilities = shifted_rewards / np.sum(shifted_rewards)
    parents_indices = np.random.choice(len(population), size=2, p=probabilities)
    return population[parents_indices[0]], population[parents_indices[1]]

# Perform crossover between two parents
def crossover(parent1, parent2):
    crossover_point = np.random.randint(1, 23)
    child1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
    child2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
    return child1, child2

# Mutate a child's pump schedule
def mutate(child):
    for i in range(len(child)):
        if np.random.rand() < MUTATION_RATE:
            child[i] = 1 - child[i]
    return child

# Run the genetic algorithm
population = initialize_population(POPULATION_SIZE)
best_schedule = None
best_reward = -np.inf

for generation in range(NUM_GENERATIONS):
    rewards = np.array([calculate_reward(individual, initial_state) for individual in population])
    
    if rewards.max() > best_reward:
        best_reward = rewards.max()
        best_schedule = population[rewards.argmax()]
    
    new_population = []
    
    for _ in range(POPULATION_SIZE // 2):
        parent1, parent2 = select_parents(population, rewards)
        child1, child2 = crossover(parent1, parent2)
        new_population.extend([mutate(child1), mutate(child2)])
    
    population = new_population

print(f"Best schedule: {best_schedule}")
print(f"Best reward: {best_reward}")

Best schedule: [1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1]
Best reward: 1228.1697436294066
